In [8]:
# importando bibliotecas
library('httr')
library("jsonlite")

In [9]:
# montando a carteira. Depois vamos pegar de algum lugar ou montar uma carteira mais real
carteiraJSON <- '[
  {"symbol" : "ALPA4", "Position" : 140900, "Price" : 12.22},
  {"symbol" : "ALSC3", "Position" : 49538,  "Price" : 14.88},
  {"symbol" : "COCE5", "Position" : 108316, "Price" : 42.70},
  {"symbol" : "ITSA4", "Position" : 534292, "Price" : 9.49}
]'

carteira <- fromJSON(carteiraJSON)

In [10]:
#pegando histórico de variação dos precos
query <- "http://api.informativos.io/prices_matrix?start_date=2016-0-01&end_date=2018-08-24&symbols[]=ALPA4"
response <- content(GET(url=query), "text")

# transformando num dataframe
data <- as.data.frame(fromJSON(response, simplifyDataFrame = TRUE), stringsAsFactors=FALSE)

#ajustando os dados
normalize_data <- function(df) {
  names(df) <- as.character(unlist(df[1,]))
  df[-1,]
}
precos <- normalize_data(data)

#criando estrutura que irá guardar os dados aplicados
precos_aplicados <- data.frame("Data" = c(), "Ativo" = c(), "Financeiro" = c(), stringsAsFactors = FALSE)

# iterando sobre a carteira
for (linha_carteira in 1:nrow(carteira)) {
    ativo <- carteira[linha_carteira, "symbol"]
    preco_original <- carteira[linha_carteira, "Price"]
    posicao_original <- carteira[linha_carteira, "Position"]
    financeiro_original <- preco_original * posicao_original
    #iterando sobre os precos
    for (linha_preco in 1:nrow(precos)) {
        # Aplicando na carteira
        change <- sapply(precos[linha_preco, ativo], as.numeric)
        perda_e_ganho <- financeiro_original * change        
        precos_aplicados <- rbind(precos_aplicados, data.frame("Data"=precos[linha_preco, "Date"], "Ativo"=ativo, "Financeiro"=perda_e_ganho))
    }    
}

# ajustando indices do dataframe
rownames(precos_aplicados) <- 1:nrow(precos_aplicados)

#agrupando por dia e somando os retornos
perda_ganho_agrupado <- aggregate(Financeiro ~ Data,precos_aplicados,sum);

print(perda_ganho_agrupado)

#ordenando os dados
ordenado <- perda_ganho_agrupado[order(perda_ganho_agrupado$Financeiro),]

print(ordenado)

#calculando o Var de 95% e 99%
var <- quantile(ordenado$Financeiro, probs=c(0.05, 0.01)) 

#imprimindo resultado
print(var)




          Data  Financeiro
1   2018-01-02   82983.954
2   2018-01-03 -235859.886
3   2018-01-04  -11244.962
4   2018-01-05  168871.137
5   2018-01-08 -193629.226
6   2018-01-09   67745.965
7   2018-01-10 -263489.315
8   2018-01-11  -11526.925
9   2018-01-12  219273.401
10  2018-01-15   50779.778
11  2018-01-16   22450.698
12  2018-01-17   95194.084
13  2018-01-18  -11089.948
14  2018-01-19 -366389.189
15  2018-01-22 -219273.401
16  2018-01-23  -88566.972
17  2018-01-24  -65550.494
18  2018-01-26  239995.028
19  2018-01-29 -117086.719
20  2018-01-30  -82966.974
21  2018-01-31 -101618.947
22  2018-02-01  102700.000
23  2018-02-02 -209215.480
24  2018-02-05  -61100.000
25  2018-02-06  -79935.924
26  2018-02-07  -68202.826
27  2018-02-08   31221.863
28  2018-02-09  236521.212
29  2018-02-14   54678.540
30  2018-02-15  -78535.294
31  2018-02-16   97451.609
32  2018-02-19  -18089.569
33  2018-02-20  253729.412
34  2018-02-21   58864.634
35  2018-02-22  275517.107
36  2018-02-23   86451.940
3

       5%        1% 
-342327.1 -581855.1 
